In [3]:
# Extract Top15 SNPs
import pandas as pd
import os
def Select_Top15_SNPs():
    raw_Traindata = pd.read_csv(Traindata, index_col=0)
    raw_Predictdata = pd.read_csv(Predictdata, index_col=0)
    Top15_SNPs = pd.read_csv(Consistent_Genes, index_col=0)
    # Top15 Features
    features_to_select = Top15_SNPs['Feature'].head(15)
    # Extract Top15 Features from raw_Traindata and raw_Predictdata
    # last column of raw_Traindata
    last_column_raw_Traindata = raw_Traindata.iloc[:, -1]
    # concat Feature column and lable 
    Top15_Traindata = pd.concat([raw_Traindata[features_to_select], last_column_raw_Traindata], axis=1)
    Top15_Traindata.index = raw_Traindata.index     
    Top15_Traindata.to_csv(f'../Data/{antb}_Top15.csv', index=True)

    last_column_raw_Predictdata = raw_Predictdata.iloc[:, -1]
    Top15_Predictdata = pd.concat([raw_Predictdata[features_to_select], last_column_raw_Predictdata], axis=1)
    Top15_Predictdata.index = raw_Predictdata.index  

    Top15_Predictdata.to_csv(f'../Data/{antb}_Top15_to_predict_MTB.csv', index=True)

#antibiotics_model
antb_Classifiers = {'rifampicin': 'GBC'}

for antb, Classifiers in antb_Classifiers.items():
    
    Traindata = f'../Data/{antb}.csv'
    Predictdata = f'../Data/{antb}_to_predict_MTB.csv'
    Consistent_Genes =f'../Figure2/All_fold6_rst/MTB_{antb}_Consistent_Genes_Per_6-fold_CV.csv'
    Select_Top15_SNPs()


In [4]:
import logging
import sys
from tqdm import tqdm
import time 


def Top15_ML_run():
    logger.info("Start runing...")
    logger.info("Importing...")
    import warnings
    warnings.filterwarnings('ignore')
    import pandas as pd
    import numpy as np
    np.random.seed(1)
    import sklearn
    import sklearn.model_selection
    import sklearn.metrics
    import sklearn.ensemble
    import seaborn as sns
    sns.set()
    import matplotlib.pyplot as plt
    from matplotlib import rc
    # %matplotlib inline

    #Result out
    file_all_out = f"{bacteria}_{antb}_Complete_Results_{validation_no}-fold_CV.csv"
    logger.info("Start Reading Data.")
    data = pd.read_csv(file_name)
    logger.info("Reading Data Done.")

    #Split data into features and labels
    X = data.iloc[:, 1:-1] 
    Y = data.iloc[:,-1] # last column label

    #Label size and matrix size
    All_Set_Data_size = data.groupby(antb).size()
    All_Set_Matrix_size = data.shape

    logger.info("Start import classifiers...")
    #Import classifiers
    from sklearn import model_selection
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.svm import SVC
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.ensemble import ExtraTreesClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import BaggingClassifier
    from sklearn.metrics import roc_curve, roc_auc_score
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import f1_score
    from sklearn.metrics import auc
    from sklearn.metrics import precision_score, recall_score
    import pickle
    from sklearn.model_selection import StratifiedKFold
    from sklearn.model_selection import LeaveOneOut 
    from sklearn.model_selection import cross_val_score
    logger.info("Import classifiers Done.")
    #Create dataframes for outputs
    Training_Performance = pd.DataFrame(columns=[])
    Test_Performance = pd.DataFrame(columns=[])
    Tf_CV = pd.DataFrame(columns=[])
    Area_Under_ROC = pd.DataFrame(columns=[])
    Area_Under_Precision_Recall = pd.DataFrame(columns=[])
    Model_Predict = pd.DataFrame(columns=[])

    # Split data into 6 equal parts
    skf = StratifiedKFold(n_splits=validation_no, random_state=42, shuffle=True)
    i = 0
    for train_index, test_index in tqdm(skf.split(X, Y), desc="StratifiedKFold"):
        logger.info("Start StratifiedKFold ind: {}.".format(i))
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

        #Build and evaluate models
        models = []
        models.append(('LogR', LogisticRegression()))
        models.append(('gNB', GaussianNB()))
        models.append(('SVM', SVC(kernel = 'rbf', probability=True)))
        models.append(('DT', DecisionTreeClassifier(criterion = 'entropy', random_state=1)))
        models.append(('RF', RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=0)))
        models.append(('KNN', KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)))
        models.append(('LDA', LinearDiscriminantAnalysis()))
        models.append(('mNB', MultinomialNB()))
        models.append(('ABC', AdaBoostClassifier()))
        models.append(('GBC', GradientBoostingClassifier()))
        models.append(('ETC', ExtraTreesClassifier()))
        models.append(('BC', BaggingClassifier()))
        logger.info("Build models done.")

 

        #Training performances
        myDF1 = pd.DataFrame(columns=[])
                #Test performances
        myDF2 = pd.DataFrame(columns=[])
        
        myDF3 = pd.DataFrame(columns=[])
        for name, model in tqdm(models, desc="myDF3"):  
            logger.info("Start kfold {}...".format(name))
            kfold = model_selection.KFold(n_splits=10, random_state=42, shuffle=True)
            logger.info("10fold model: {} done. ".format(name))
            results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold)
            mean= results.mean().round(3)
            std = results.std()
            logger.info("Model {} 10fold: mean:{},\t std:{}.".format(name, mean, std))
            myDF3 = myDF3._append({'classifier': name, f'ten_f_CV{i+1}':mean}, ignore_index = True)
        Tf_CV = pd.concat([Tf_CV, myDF3], axis = 1)

        myDF4 = pd.DataFrame(columns=[])
        myDF5 = pd.DataFrame(columns=[])
        for name, model in tqdm(models, desc="myDF12"):  
            logger.info("Start training {}...".format(name))  
            model = model.fit(X_train, Y_train)
            logger.info("Train model: {} done. Then start predicting X_train...".format(name))
            Y_train_pred = model.predict(X_train)
            Y_test_pred = model.predict(X_test)
            logger.info("{} predict done.".format(name))
            Tr_precision = precision_score(Y_train, Y_train_pred, average='macro').round(3)
            Tr_recall = recall_score(Y_train, Y_train_pred, average='macro').round(3)
            Tr_f1 = f1_score (Y_train, Y_train_pred, average='macro').round(3)
            logger.info("Model {} X_train: Tr_precision:{},\t Tr_recall:{},\t Tr_f1:{}.".format(name, Tr_precision, Tr_recall, Tr_f1))
            myDF1 = myDF1._append({'classifier': name, f'tr_precision{i+1}': Tr_precision, f'tr_recall{i+1}': Tr_recall, f'tr_f1 {i+1}':Tr_f1}, ignore_index = True)
            # report = sklearn.metrics.classification_report(Y_test, Y_test_pred)
            Te_precision = precision_score(Y_test, Y_test_pred, average='macro').round(3)
            Te_recall = recall_score(Y_test, Y_test_pred, average='macro').round(3)
            Te_f1 = f1_score (Y_test, Y_test_pred, average='macro').round(3)
            logger.info("Model {} X_train: Te_precision:{},\t Te_recall:{},\t Te_f1:{}.".format(name, Te_precision, Te_recall, Te_f1))
            myDF2 = myDF2._append({'classifier': name, f'te_precision{i+1}': Te_precision, f'te_recall{i+1}': Te_recall, f'te_f1 {i+1}':Te_f1}, ignore_index = True)
            y_pred_proba = model.predict_proba(X_test)[::,1]
            logger.info("y_pred_proba: {} done. ".format(name))
            fpr, tpr, _ = roc_curve(Y_test,  y_pred_proba, pos_label = None)
            a_u_c = roc_auc_score(Y_test, y_pred_proba).round(3)
            logger.info("Model {} AU_ROC: fpr:{},\t tpr:{}.".format(name, fpr, tpr))
            myDF4 = myDF4._append({'a classifier': name, f'au ROC {i+1}': a_u_c}, ignore_index = True)
            y_pred_proba5 = model.predict_proba(X_test)
            logger.info("y_pred_proba5: {} done. ".format(name))
            # keep probabilities for the positive outcome only
            y_pred_proba5 = y_pred_proba5[:, 1]
            #predict class vlaues
            y_pred5 = model.predict(X_test)
            # calculate precision-recall curve
            logger.info("Predict: {} done. calculate precision-recall curve...".format(name))
            precision, recall, _ = precision_recall_curve(Y_test, y_pred_proba5)
            logger.info("Predict: {} done. calculate au precision-recall curve...".format(name))
            # calculate au precision-recall curve
            area = auc(recall, precision).round(3)
            # calculate f1 score
            logger.info("Predict: {} done. calculate f1 score...".format(name))
            f1 = f1_score(Y_test, y_pred5).round(3)
            logger.info("Model {} AU_ROC: precision:{},\t recall:{},\t area:{},\t f1:{}.".format(name, precision, recall,area,f1))
            myDF5 = myDF5._append({'a classifier': name, f'au PR {i+1}': area}, ignore_index = True)
        Training_Performance = pd.concat([Training_Performance, myDF1], axis = 1)
        Test_Performance = pd.concat([Test_Performance, myDF2], axis = 1)
        Area_Under_ROC = pd.concat([Area_Under_ROC, myDF4], axis = 1)
        Area_Under_Precision_Recall = pd.concat([Area_Under_Precision_Recall, myDF5], axis = 1)
        i += 1

    # Tf_CV.to_csv(f'{bacteria}_{antb}_All_Set_Tf_CV_{validation_no}_Top15-fold_CV.csv', encoding='utf-8') 
    # Training_Performance.to_csv(f'{bacteria}_{antb}_All_Set_Performance_Training_Performance_{validation_no}_Top15-fold_CV.csv', encoding='utf-8') 
    # Test_Performance.to_csv(f'{bacteria}_{antb}_All_Set_Test_Training_Performance_{validation_no}_Top15-fold_CV.csv', encoding='utf-8') 
    # Area_Under_ROC.to_csv(f'{bacteria}_{antb}_All_Set_Test_Area_Under_ROC_{validation_no}_Top15-fold_CV.csv', encoding='utf-8') 
    # Area_Under_Precision_Recall.to_csv(f'{bacteria}_{antb}_All_Set_Test_Area_Under_Precision_Recall_{validation_no}_Top15-fold_CV.csv', encoding='utf-8')


    #Model names
    Models = Tf_CV.iloc[:, 0] 
    #Calculating the mean of all folds
    logger.info("Calculating the mean of all folds")
    #training f1 average Training_Performance.filter(like='tr_precision').mean(axis=1).round(3)
    tr_f1_avg = Training_Performance.filter(like='tr_f1').mean(axis=1).round(3)
    tr_f1_avg = tr_f1_avg.rename('tr_f1_avg', inplace=True)
    #Training precision average
    tr_precision_avg = Training_Performance.filter(like='tr_precision').mean(axis=1).round(3)
    tr_precision_avg = tr_precision_avg.rename('tr_precision_avg', inplace=True)

    #Training recall average
    tr_recall_avg = Training_Performance.filter(like='tr_recall').mean(axis=1).round(3)
    tr_recall_avg = tr_recall_avg.rename('tr_recall_avg', inplace=True)

    #Test f1 average
    te_f1_avg = Test_Performance.filter(like='te_f1').mean(axis=1).round(3)
    te_f1_avg = te_f1_avg.rename('te_f1_avg', inplace=True)

    #Test precision average
    te_precision_avg = Test_Performance.filter(like='te_precision').mean(axis=1).round(3)
    te_precision_avg = te_precision_avg.rename('te_precision_avg', inplace=True)

    #Test recall average
    te_recall_avg = Test_Performance.filter(like='te_recall').mean(axis=1).round(3)
    te_recall_avg = te_recall_avg.rename('te_recall_avg', inplace=True)

    #Tf_CV average
    Tf_CV_Avg = Tf_CV.filter(like='ten_f_CV').mean(axis=1).round(3)
    Tf_CV_Avg = Tf_CV_Avg.rename('Tf_CV_Avg', inplace=True)

    #Area_Under_ROC average
    au_ROC_avg = Area_Under_ROC.filter(like='au ROC').mean(axis=1).round(3)
    au_ROC_avg = au_ROC_avg.rename('au_ROC_avg', inplace=True)

    #Area_Under_Precision_Recall average
    au_PR_avg= Area_Under_Precision_Recall.filter(like='au PR').mean(axis=1).round(3)
    au_PR_avg = au_PR_avg.rename('au_PR_avg', inplace=True)

    #Accumulating all dataframes
    logger.info("Accumulating all dataframes")
    frames2 = [Models, tr_precision_avg, tr_recall_avg, tr_f1_avg, te_precision_avg, te_recall_avg, te_f1_avg, Tf_CV_Avg, au_ROC_avg, au_PR_avg]

    #Result all set
    Final_All_set_Results= pd.concat(frames2, axis=1)
    
    # Result out Modify
    logger.info("Result all set")
    with open (file_all_out, 'a+') as rf:
        rf.write(f'\n{bacteria} {antb} All set Results\n')
        rf.write(f'\n{All_Set_Data_size}\n')
        rf.write(f'\nmatrix_size: {All_Set_Matrix_size}\n\n')

        Final_All_set_Results.to_csv(rf)

    #Fit on whole set and predict the labels
    models1 = []
    models1.append(('LogR', LogisticRegression()))
    models1.append(('gNB', GaussianNB()))
    models1.append(('SVM', SVC(kernel = 'rbf', probability=True)))
    models1.append(('DT', DecisionTreeClassifier(criterion = 'entropy', random_state=1)))
    models1.append(('RF', RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=0)))
    models1.append(('KNN', KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)))
    models1.append(('LDA', LinearDiscriminantAnalysis()))
    models1.append(('mNB', MultinomialNB()))
    models1.append(('ABC', AdaBoostClassifier()))
    models1.append(('GBC', GradientBoostingClassifier()))
    models1.append(('ETC', ExtraTreesClassifier()))
    models1.append(('BC', BaggingClassifier()))

    #Predict Lables
    logger.info("Predict Lables")
    predict_lab = []
    predicted_df = pd.DataFrame(columns=[])

    for name, model in models1:
        logger.info("model pickle_files {}...".format(name))
    #Fit the model on whole dataset
        model.fit(X, Y)
        pickle_files = f'../Figure3/Top15_model/{antb}_{name}_Top15.sav'

        #Save the mode as pickle file
        pickle.dump(model, open(pickle_files, 'wb'))


#antibiotics_model
antb_Classifiers = {'rifampicin': 'GBC'}
for antb, Classifiers in antb_Classifiers.items():
    
    #Acronym for Klebsiella pneumoniae
    bacteria = 'MTB'
    
    #Italicized full name bacteria for fig output
    italic_name = r"Mycobacterium tuberculosis"
    
    #Import amr-ast data from github repository
    file_name = f'../Data/{antb}_Top15.csv'

    #Import bacterial strains without caapenemase from github repository
    To_predict = f'../Data/{antb}_Top15_to_predict_MTB.csv'
    
    #no of validation
    validation_no = 6

    # loggingfile config
    logger = logging.getLogger(antb)
    logger.setLevel(logging.INFO)
    rf_handler = logging.StreamHandler(sys.stderr)
    rf_handler.setLevel(logging.DEBUG) 
    rf_handler.setFormatter(logging.Formatter("%(asctime)s - %(name)s - %(message)s"))
    f_handler = logging.FileHandler('{}_{}.txt'.format(antb, time.strftime("%Y-%m-%d%H-%M-%S", time.localtime())))
    f_handler.setLevel(logging.INFO)
    f_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(filename)s[:%(lineno)d] - %(message)s"))
    
    logger.addHandler(rf_handler)
    logger.addHandler(f_handler)
        
    Top15_ML_run()

2024-09-03 10:56:08,478 - rifampicin - Start runing...
2024-09-03 10:56:08,478 - rifampicin - Importing...
2024-09-03 10:56:09,412 - rifampicin - Start Reading Data.
2024-09-03 10:56:09,426 - rifampicin - Reading Data Done.
2024-09-03 10:56:09,431 - rifampicin - Start import classifiers...
2024-09-03 10:56:09,436 - rifampicin - Import classifiers Done.
StratifiedKFold: 0it [00:00, ?it/s]2024-09-03 10:56:09,909 - rifampicin - Start StratifiedKFold ind: 0.
2024-09-03 10:56:09,913 - rifampicin - Build models done.

myDF3:   0%|                                                                                                                | 0/12 [00:00<?, ?it/s]2024-09-03 10:56:09,916 - rifampicin - Start kfold LogR...
2024-09-03 10:56:09,916 - rifampicin - 10fold model: LogR done. 
2024-09-03 10:56:10,071 - rifampicin - Model LogR 10fold: mean:0.959,	 std:0.0073994698456071265.

myDF3:   8%|████████▋                                                                                           

2024-08-20 16:24:27,934 - rifampicin - Start training SVM...
2024-08-20 16:24:28,260 - rifampicin - Train model: SVM done. Then start predicting X_train...
2024-08-20 16:24:28,378 - rifampicin - SVM predict done.
2024-08-20 16:24:28,384 - rifampicin - Model SVM X_train: Tr_precision:0.965,	 Tr_recall:0.96,	 Tr_f1:0.963.
2024-08-20 16:24:28,390 - rifampicin - Model SVM X_train: Te_precision:0.946,	 Te_recall:0.938,	 Te_f1:0.942.
2024-08-20 16:24:28,412 - rifampicin - y_pred_proba: SVM done. 
2024-08-20 16:24:28,416 - rifampicin - Model SVM AU_ROC: fpr:[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00391389 0.00391389 0.00391389 0.00391389
 0.00587084 0.00587084 0.00587084 0.00587084 0.00587084 0

2024-08-20 16:24:28,480 - rifampicin - DT predict done.
2024-08-20 16:24:28,485 - rifampicin - Model DT X_train: Tr_precision:0.974,	 Tr_recall:0.961,	 Tr_f1:0.967.
2024-08-20 16:24:28,490 - rifampicin - Model DT X_train: Te_precision:0.94,	 Te_recall:0.926,	 Te_f1:0.932.
2024-08-20 16:24:28,493 - rifampicin - y_pred_proba: DT done. 
2024-08-20 16:24:28,496 - rifampicin - Model DT AU_ROC: fpr:[0.         0.01956947 0.01956947 0.01956947 0.02348337 0.02348337
 0.02739726 0.02739726 0.02739726 0.03913894 0.03913894 0.04109589
 0.05870841 0.06457926 0.0665362  0.0704501  0.08610568 0.09197652
 0.11937378 0.13502935 0.14285714 0.15655577 0.22700587 0.24853229
 0.30136986 0.34050881 0.36399217 0.56164384 0.76712329 0.8297456
 1.        ],	 tpr:[0.         0.81067961 0.82038835 0.83009709 0.83495146 0.84466019
 0.8592233  0.87378641 0.87864078 0.89320388 0.89805825 0.89805825
 0.89805825 0.90776699 0.91262136 0.91262136 0.91262136 0.91262136
 0.91262136 0.9223301  0.9223301  0.92718447 0.936

2024-08-20 16:24:28,925 - rifampicin - Model LDA AU_ROC: fpr:[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00587084 0.00587084 0.00587084
 0.00782779 0.00978474 0.00978474 0.01174168 0.01174168 0.01369863
 0.01369863 0.01565558 0.01761252 0.01956947 0.01956947 0.02152642
 0.02152642 0.02544031 0.02544031 0.02739726 0.02739726 0.02935421
 0.03131115 0.03131115 0.03522505 0.037182   0.037182   0.03913894
 0.04305284 0.048

2024-08-20 16:24:28,961 - rifampicin - y_pred_proba5: mNB done. 
2024-08-20 16:24:28,962 - rifampicin - Predict: mNB done. calculate precision-recall curve...
2024-08-20 16:24:28,963 - rifampicin - Predict: mNB done. calculate au precision-recall curve...
2024-08-20 16:24:28,964 - rifampicin - Predict: mNB done. calculate f1 score...
2024-08-20 16:24:28,967 - rifampicin - Model mNB AU_ROC: precision:[0.28730823 0.28851541 0.2883263  0.28913963 0.29955947 0.30104322
 0.30149254 0.3019432  0.30606061 0.30652504 0.3093415  0.31162791
 0.31357254 0.33001658 0.33056478 0.33614865 0.33671743 0.33843537
 0.33788396 0.33962264 0.33907057 0.34751773 0.34875445 0.35571688
 0.35636364 0.35766423 0.35714286 0.36380597 0.36448598 0.45348837
 0.45774648 0.45882353 0.47445255 0.47677262 0.47794118 0.47665848
 0.48019802 0.48138958 0.52291105 0.52432432 0.52574526 0.53739612
 0.53611111 0.53760446 0.53651685 0.53802817 0.53954802 0.54261364
 0.54415954 0.56342183 0.56508876 0.56676558 0.56716418 0.568

...

2024-08-20 16:25:15,904 - rifampicin - Model LDA X_train: Tr_precision:0.957,	 Tr_recall:0.913,	 Tr_f1:0.932.
2024-08-20 16:25:15,909 - rifampicin - Model LDA X_train: Te_precision:0.967,	 Te_recall:0.927,	 Te_f1:0.945.
2024-08-20 16:25:15,912 - rifampicin - y_pred_proba: LDA done. 
2024-08-20 16:25:15,916 - rifampicin - Model LDA AU_ROC: fpr:[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.00195695 0.00195695 0.00195695
 0.00195695 0.00195695 0.00195695 0.00391389 0.00391389 0.00391389
 0.00391389 0.00587084 0.00587084 0.00782779 0.01369863 0.01369863
 0.01761252 0.02152642 0.02544031 0.02739

2024-08-20 16:25:15,953 - rifampicin - y_pred_proba5: mNB done. 
2024-08-20 16:25:15,955 - rifampicin - Predict: mNB done. calculate precision-recall curve...
2024-08-20 16:25:15,957 - rifampicin - Predict: mNB done. calculate au precision-recall curve...
2024-08-20 16:25:15,957 - rifampicin - Predict: mNB done. calculate f1 score...
2024-08-20 16:25:15,961 - rifampicin - Model mNB AU_ROC: precision:[0.28631285 0.28751753 0.28792135 0.28913963 0.28954802 0.29927007
 0.30014641 0.3010279  0.30163447 0.30253353 0.30343797 0.30526316
 0.30699088 0.30745814 0.30792683 0.30839695 0.32679739 0.33333333
 0.33500838 0.33613445 0.33670034 0.33726813 0.33783784 0.33955857
 0.34246575 0.34364261 0.35149385 0.35211268 0.35971223 0.36297641
 0.36363636 0.36247723 0.36666667 0.36734694 0.45496536 0.45601852
 0.45707657 0.46028037 0.47228916 0.47457627 0.47688564 0.47804878
 0.4792176  0.48157248 0.48275862 0.48129676 0.4825     0.51604278
 0.51742627 0.5188172  0.53168044 0.53611111 0.53501401 0.541

2024-08-20 16:25:16,129 - rifampicin - Predict: ABC done. calculate f1 score...
2024-08-20 16:25:16,132 - rifampicin - Model ABC AU_ROC: precision:[0.28631285 0.28671329 0.28895184 0.2893617  0.29101284 0.34228188
 0.34517766 0.34693878 0.34752981 0.34871795 0.42379958 0.43939394
 0.44395604 0.45701357 0.48557692 0.48792271 0.48910412 0.49029126
 0.54200542 0.55555556 0.56022409 0.56338028 0.58309038 0.58651026
 0.58823529 0.59171598 0.59347181 0.5988024  0.6006006  0.621875
 0.62578616 0.6503268  0.65245902 0.65460526 0.65676568 0.71071429
 0.71326165 0.72627737 0.73431734 0.73977695 0.74531835 0.7481203
 0.75665399 0.76538462 0.76833977 0.77131783 0.78656126 0.78968254
 0.79282869 0.796      0.80241935 0.80566802 0.80894309 0.8122449
 0.83122363 0.83050847 0.83760684 0.84482759 0.84415584 0.84782609
 0.85152838 0.85526316 0.87782805 0.88127854 0.8853211  0.89767442
 0.90186916 0.91904762 0.92788462 0.92753623 0.93203883 0.93627451
 0.93596059 0.93564356 0.94416244 0.94897959 0.948717


myDF12:  83%|██████████████████████████████████████████████████████████████████████████████▎               | 10/12 [00:01<00:00,  7.06it/s]2024-08-20 16:25:16,327 - rifampicin - Start training ETC...
2024-08-20 16:25:16,464 - rifampicin - Train model: ETC done. Then start predicting X_train...
2024-08-20 16:25:16,508 - rifampicin - ETC predict done.
2024-08-20 16:25:16,514 - rifampicin - Model ETC X_train: Tr_precision:0.971,	 Tr_recall:0.958,	 Tr_f1:0.964.
2024-08-20 16:25:16,520 - rifampicin - Model ETC X_train: Te_precision:0.965,	 Te_recall:0.959,	 Te_f1:0.962.
2024-08-20 16:25:16,532 - rifampicin - y_pred_proba: ETC done. 
2024-08-20 16:25:16,537 - rifampicin - Model ETC AU_ROC: fpr:[0.         0.00587084 0.00587084 0.00587084 0.00587084 0.00587084
 0.00587084 0.00587084 0.00587084 0.00587084 0.00587084 0.00587084
 0.00587084 0.00782779 0.00782779 0.00782779 0.00782779 0.01174168
 0.01174168 0.01369863 0.01369863 0.01369863 0.01761252 0.01761252
 0.01956947 0.02348337 0.02544031 

2024-08-20 16:25:16,676 - rifampicin - model pickle_files gNB...
2024-08-20 16:25:16,679 - rifampicin - model pickle_files SVM...
2024-08-20 16:25:17,159 - rifampicin - model pickle_files DT...
2024-08-20 16:25:17,165 - rifampicin - model pickle_files RF...
2024-08-20 16:25:17,186 - rifampicin - model pickle_files KNN...
2024-08-20 16:25:17,202 - rifampicin - model pickle_files LDA...
2024-08-20 16:25:17,212 - rifampicin - model pickle_files mNB...
2024-08-20 16:25:17,216 - rifampicin - model pickle_files ABC...
2024-08-20 16:25:17,314 - rifampicin - model pickle_files GBC...
2024-08-20 16:25:17,484 - rifampicin - model pickle_files ETC...
2024-08-20 16:25:17,642 - rifampicin - model pickle_files BC...


In [5]:
#reset the column of India independent dataset
import pandas as pd

# a.csv 
df_a = pd.read_csv('../Figure3/Data_to_predict/India_rifampicin_to_predict_MTB_166.csv', index_col=0)

# b.csv 
df_b = pd.read_csv('../Data/rifampicin_Top15_to_predict_MTB.csv', index_col=0)

#b.csv a.csv 
reordered_df_a = df_a.reindex(columns=df_b.columns)

# save c.csv 
reordered_df_a.to_csv('../Figure3/Data_to_predict/India_rifampicin_to_predict_MTB_166_sort.csv',index=True)



In [12]:
# To predict in independent India dataset
#Load all models and to predict phenotypes in independent India dataset
import logging
import sys
from tqdm import tqdm
import time 

import pandas as pd
import numpy as np

import shap
import xgboost
import matplotlib.pyplot as plt

def To_predict_ML_run():
    import warnings
    from sklearn import model_selection
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.svm import SVC
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    from sklearn.ensemble import ExtraTreesClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import BaggingClassifier
    from sklearn.metrics import roc_curve, roc_auc_score
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import f1_score
    from sklearn.metrics import auc
    from sklearn.metrics import precision_score, recall_score
    import pickle
    from sklearn.model_selection import StratifiedKFold
    from sklearn.model_selection import LeaveOneOut 
    from sklearn.model_selection import cross_val_score

    warnings.filterwarnings('ignore')
    models1 = []
    models1.append(('LogR', LogisticRegression()))
    models1.append(('gNB', GaussianNB()))
    models1.append(('SVM', SVC(kernel = 'rbf', probability=True)))
    models1.append(('DT', DecisionTreeClassifier(criterion = 'entropy', random_state=1)))
    models1.append(('RF', RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=0)))
    models1.append(('KNN', KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)))
    models1.append(('LDA', LinearDiscriminantAnalysis()))
    models1.append(('mNB', MultinomialNB()))
    models1.append(('ABC', AdaBoostClassifier()))
    models1.append(('GBC', GradientBoostingClassifier()))
    models1.append(('ETC', ExtraTreesClassifier()))
    models1.append(('BC', BaggingClassifier()))

    #Predict Lables
    # logger.info("Predict Lables")
    predict_lab = []
    predicted_df = pd.DataFrame(columns=[])

    for name, model in models1:

        pickle_files = f'../Figure3/Top15_model/{antb}_{name}_Top15.sav'
        #Load the model from disk
        loaded_model = pickle.load(open(pickle_files, 'rb'))
        #Predicting the new data
        pr = pd.read_csv(Independent_to_predict)
        Xnew = pr.iloc[:, 1:-1]
        ynew = loaded_model.predict(Xnew)
        predict_lab.append({'predicted label':ynew})
        labels = pd.DataFrame(data=ynew, columns=[f'prediction by {name}'])
        predicted_df = pd.concat([predicted_df, labels], axis=1)
        #Labels predicted my each model    
    Model_Predict = pd.DataFrame(predict_lab)

    #Separate df for each isolate and predicted label by models
    Predicted_Labels = pd.concat([pr['Isolate'], predicted_df], axis=1)  
    file_all_out = file_all_outfile
    
    with open (file_all_out, 'w') as rf:
        Predicted_Labels.to_csv(rf, index=False)


        
#antibiotics_model
antb_Classifiers = {'rifampicin': 'GBC'}
for antb, Classifiers in antb_Classifiers.items():
    #Import amr-ast data from github repository
    Independent_to_predict = f'../Figure3/Data_to_predict/India_{antb}_to_predict_MTB_166_sort.csv'
    
    file_all_outfile = f'../Figure3/Data_to_predict_rst/rst_India_{antb}_to_predict_MTB.csv'
    
    To_predict_ML_run()

In [13]:
#Independant Model performance Summary
import pandas as pd

df_a = pd.read_csv('../Figure3/Data_to_predict_rst/rst_India_rifampicin_to_predict_MTB.csv', index_col=0).dropna()

df_b = pd.read_csv('../Figure3/Data_to_predict/India_rifampicin_Top15_phenotype.txt', index_col=0, sep='\t')
df_b.iloc[:, -1].replace({'R': 1, 'S': 0}, inplace=True)

merged_df = df_a.join(df_b.iloc[:, -1], on=df_a.index)

# merged_df.to_csv('../Figure3/Data_to_predict_rst/rifampicin_merged_result.csv')

merged_df.iloc[:,5:13]
def calculate_metrics(y_true, y_pred, label):

    TP = sum((y_true == 1) & (y_pred == 1))
    FP = sum((y_true == 0) & (y_pred == 1))
    TN = sum((y_true == 0) & (y_pred == 0))
    FN = sum((y_true == 1) & (y_pred == 0))

    accuracy = (TP + TN) / (TP + FP + TN + FN) if (TP + FP + TN + FN) != 0 else 0
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return {f'{label}_TP': TP, f'{label}_FP': FP, f'{label}_TN': TN, f'{label}_FN': FN,
            f'{label}_Accuracy': accuracy, f'{label}_Precision': precision,
            f'{label}_Recall': recall, f'{label}_F1_Score': f1_score}

results = {}
for col in merged_df.columns[0:12]:
    y_true = merged_df['AMR']
    y_pred = merged_df[col]
    results.update(calculate_metrics(y_true, y_pred, col.replace('prediction by ', '')))

results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Value'])
results_df.to_csv(f'../Figure3/Data_to_predict_rst/rifampicin_predict_rst_12model.csv', index=True)
